In [1]:
import word_score
text="It was very good"
score=word_score.get_tuple(text)
score

(0.58, 0.0)

In [2]:
import csv
rulebase=[]
with open("nominal_rulebase.csv") as csv_file:
    reader=csv.reader(csv_file)
    rulebase.append(next(reader))
    for row in reader:
        c=row[-1]
        
        row=[[float(i) for i in val.split('[')[1].split(']')[0].split(',')] for val in row[:-1]]+[row[-1]]
        rulebase.append(row)
rulebase[:]

[['pos_score', 'neg_score', 'class'],
 [[0.28, 0.42000000000000004], [0.0, 0.125], 'g'],
 [[0.28, 0.42000000000000004], [0.0, 0.125], 'g'],
 [[0.14, 0.28], [0.0, 0.125], 'g'],
 [[0.42000000000000004, 0.56], [0.0, 0.125], 'g'],
 [[0.42000000000000004, 0.56], [0.125, 0.25], 'g'],
 [[0.42000000000000004, 0.56], [0.0, 0.125], 'g'],
 [[0.42000000000000004, 0.56], [0.0, 0.125], 'g'],
 [[0.14, 0.28], [0.0, 0.125], 'g'],
 [[0.28, 0.42000000000000004], [0.0, 0.125], 'g'],
 [[0.28, 0.42000000000000004], [0.125, 0.25], 'g'],
 [[0.28, 0.42000000000000004], [0.0, 0.125], 'g'],
 [[0.28, 0.42000000000000004], [0.0, 0.125], 'g'],
 [[0.0, 0.14], [0.125, 0.25], 'b'],
 [[0.0, 0.14], [0.25, 0.375], 'b'],
 [[0.0, 0.14], [0.375, 0.5], 'b'],
 [[0.0, 0.14], [0.125, 0.25], 'b'],
 [[0.0, 0.14], [0.125, 0.25], 'b'],
 [[0.0, 0.14], [0.25, 0.375], 'b'],
 [[0.14, 0.28], [0.25, 0.375], 'b'],
 [[0.0, 0.14], [0.125, 0.25], 'b'],
 [[0.0, 0.14], [0.375, 0.5], 'b'],
 [[0.0, 0.14], [0.25, 0.375], 'b'],
 [[0.0, 0.14], [0.2

In [11]:
def get_prob(i,value,t):
    count=0
    hit=0
    #print("\n\n",i,value,t)
    for row in rulebase[1:]:
        #print(row[i],row[-1])
        if row[-1]==t:
            count+=1
            #print('count:',count)
        if row[-1]==t and row[i]==value:
            hit+=1
    print("i:",i,'value:',value,"class:",t,'hit:',hit)
    #print(round(float(hit/count),2))    
    return round(float(hit/count),2)

In [12]:
def prep_tuple(t):
    result=[]
    for i in range(len(t)):
        values=[row[i] for row in rulebase[1:]]
        values.sort()
        #print ("\n",values)
        if t[i]>values[-1][1]:
            result.append(values[-1])
            continue
        for j in values:
            if t[i]>=j[0] and t[i]<=j[1]:
                #print(j,"\n\n")
                result.append(j)
                break
            
    return result

In [13]:
import pprint as pp
def nb(tup):
    targets=set([row[-1] for row in rulebase[1:]])
    target_score={}
    target_prob={}
    #print(targets)
    target_column=[row[-1] for row in rulebase[1:]]
    for t in targets:
        target_score[t]=0
        target_prob[t]=float(target_column.count(t)/len(target_column))
    
    probs={}
        
    for i in range(len(rulebase[0][:-1])):
        probs[i]={}
        active=probs[i]
        column=[row[i] for row in rulebase[1:]]
        for value in column:
            if str(value) not in active:
                active[str(value)]={}
            
                for t in targets:
                    active[str(value)][t]=get_prob(i,value,t)
    pp.pprint (probs)        
    
    
    
    #print("\n\n",tup)
    tup=prep_tuple(tup)
    
    #print("\n\n",tup)
    for t in targets:
        target_score[t]=target_prob[t]
        for i in range(len(tup)):
            target_score[t]*=probs[i][str(tup[i])][t]
    
    #print(target_score)
    import operator
    output=max(target_score.items(), key=operator.itemgetter(1))[0]
    return output


In [14]:
nb(score)

i: 0 value: [0.28, 0.42000000000000004] class: g hit: 6
i: 0 value: [0.28, 0.42000000000000004] class: b hit: 0
i: 0 value: [0.28, 0.42000000000000004] class: n hit: 0
i: 0 value: [0.14, 0.28] class: g hit: 2
i: 0 value: [0.14, 0.28] class: b hit: 3
i: 0 value: [0.14, 0.28] class: n hit: 11
i: 0 value: [0.42000000000000004, 0.56] class: g hit: 4
i: 0 value: [0.42000000000000004, 0.56] class: b hit: 0
i: 0 value: [0.42000000000000004, 0.56] class: n hit: 0
i: 0 value: [0.0, 0.14] class: g hit: 0
i: 0 value: [0.0, 0.14] class: b hit: 14
i: 0 value: [0.0, 0.14] class: n hit: 10
i: 1 value: [0.0, 0.125] class: g hit: 10
i: 1 value: [0.0, 0.125] class: b hit: 0
i: 1 value: [0.0, 0.125] class: n hit: 17
i: 1 value: [0.125, 0.25] class: g hit: 2
i: 1 value: [0.125, 0.25] class: b hit: 4
i: 1 value: [0.125, 0.25] class: n hit: 4
i: 1 value: [0.25, 0.375] class: g hit: 0
i: 1 value: [0.25, 0.375] class: b hit: 11
i: 1 value: [0.25, 0.375] class: n hit: 0
i: 1 value: [0.375, 0.5] class: g hit: 0

'g'